# Install TileDB.CSharp

In [ ]:
#r "nuget:TileDB.CSharp"

In [ ]:
using TileDB.CSharp;

# Helper class

In [ ]:
using System;
using System.IO;
public class Helper {

    public static void CreateTestArray(string uri)
    {
        string tmpArrayPath = uri;
        if (System.IO.Directory.Exists(tmpArrayPath))
        {
            System.IO.Directory.Delete(tmpArrayPath, true);
        }

        var context = TileDB.CSharp.Context.GetDefault();
        var domain = new TileDB.CSharp.Domain(context);

        var dim_1 = TileDB.CSharp.Dimension.Create(context, "d1", new[] { 1, 1 }, 1);
        domain.AddDimension(dim_1);

        var array_schema = new TileDB.CSharp.ArraySchema(context, TileDB.CSharp.ArrayType.TILEDB_DENSE);

        var attr1 = new TileDB.CSharp.Attribute(context, "a1", TileDB.CSharp.DataType.TILEDB_FLOAT32);
        array_schema.AddAttribute(attr1);

        array_schema.SetDomain(domain);
        array_schema.SetCellOrder(TileDB.CSharp.LayoutType.TILEDB_ROW_MAJOR);
        array_schema.SetTileOrder(TileDB.CSharp.LayoutType.TILEDB_ROW_MAJOR);

        array_schema.Check();

        TileDB.CSharp.Array.Create(context, tmpArrayPath, array_schema);
    }
}

# Group 

## Create temp directory

In [ ]:
var ctx=TileDB.CSharp.Context.GetDefault();
var temp_dir = System.IO.Path.Combine(System.IO.Directory.GetCurrentDirectory(),"tiledb_test_group");
using(var vfs = new TileDB.CSharp.VFS(ctx)) {
    if(vfs.IsDir(temp_dir))
    {
        vfs.RemoveDir(temp_dir);
    }
    vfs.CreateDir(temp_dir);
}

## Create arrays and groups

In [ ]:
string array1_uri = System.IO.Path.Combine(temp_dir,"array1");
string array2_uri = System.IO.Path.Combine(temp_dir,"array2");
string array3_uri = System.IO.Path.Combine(temp_dir,"array3");
Helper.CreateTestArray(array1_uri);
Helper.CreateTestArray(array2_uri);
Helper.CreateTestArray(array3_uri);
string group1_uri = System.IO.Path.Combine(temp_dir,"group1");
string group2_uri = System.IO.Path.Combine(temp_dir,"group2");
TileDB.CSharp.Group.Create(ctx,group1_uri);
TileDB.CSharp.Group.Create(ctx,group2_uri);

## Add members to groups

In [ ]:
var group1 = new TileDB.CSharp.Group(ctx,group1_uri);
group1.Open(TileDB.CSharp.QueryType.TILEDB_WRITE);
var group2 = new TileDB.CSharp.Group(ctx,group2_uri);
group2.Open(TileDB.CSharp.QueryType.TILEDB_WRITE);
group1.AddMember(array1_uri,false);
group1.AddMember(array2_uri,false);
group2.AddMember(array3_uri,false);
group1.Close();
group2.Close();

## Group metadata

In [ ]:
var group = new TileDB.CSharp.Group(ctx,group1_uri);
group.Open(TileDB.CSharp.QueryType.TILEDB_WRITE);
group.PutMetadata<int>("key1",10);
group.Close();
group.Open(TileDB.CSharp.QueryType.TILEDB_READ);
var v1 = group.GetMetadata<int>("key1");
System.Console.WriteLine("v1:{0}", v1);